In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import time

In [2]:
data = np.load("portfolio_qubo_data.npz")

Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)

print("Data loaded successfully.")
print(f"n = {n} (assets)")
print(f"B = {B} (cardinality)")
print(f"Q shape: {Q.shape}")
print(f"q shape: {q.shape}")

Data loaded successfully.
n = 21 (assets)
B = 4 (cardinality)
Q shape: (21, 21)
q shape: (21,)


In [3]:
print("Formulating CVXPy problem.")

x = cp.Variable(n, boolean=True)
objective_func = cp.quad_form(x, Q) + q.T @ x
objective = cp.Minimize(objective_func)

constraints = [cp.sum(x) == B]

problem = cp.Problem(objective, constraints)

print("Solving with SCIP Solver.")
t0 = time.perf_counter()

problem.solve(solver='SCIP', verbose=True)
t1 = time.perf_counter()
print(f"\nSolve time: {(t1 - t0):.4f} seconds")

(CVXPY) Nov 07 05:08:13 PM: Your problem has 21 variables, 1 constraints, and 0 parameters.
(CVXPY) Nov 07 05:08:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 07 05:08:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)


Formulating CVXPy problem.
Solving with SCIP Solver.
                                     CVXPY                                     
                                     v1.7.3                                    


(CVXPY) Nov 07 05:08:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 07 05:08:13 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 07 05:08:13 PM: Compiling problem (target solver=SCIP).
(CVXPY) Nov 07 05:08:13 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIP
(CVXPY) Nov 07 05:08:13 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 07 05:08:13 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 07 05:08:13 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Nov 07 05:08:13 PM: Applying reduction SCIP
(CVXPY) Nov 07 05:08:13 PM: Finished problem compilation (took 1.432e-01 seconds).
(CVXPY) Nov 07 05:08:13 PM: Invoking solver SCIP  to obtain a solution.


-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
presolving:
(round 1, fast)       2 del vars, 2 del conss, 0 add conss, 45 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.1s) probing cycle finished: starting next cycle
   (0.1s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.1s) no symmetry present (symcode time: 0.00)
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 2 deleted vars, 2 deleted constraints, 0 added constraints, 45 tightened bounds, 0 added holes, 0 changed sides

(CVXPY) Nov 07 05:08:14 PM: Problem status: optimal
(CVXPY) Nov 07 05:08:14 PM: Optimal value: -1.198e-03
(CVXPY) Nov 07 05:08:14 PM: Compilation took 1.432e-01 seconds
(CVXPY) Nov 07 05:08:14 PM: Solver (including time spent in interface) took 1.502e+00 seconds


  1.1s|     1 |     0 |   972 |     - |  2497k |   0 |  56 |  27 | 141 | 109 | 11 |   4 |   4 |-1.258987e-03 |-1.197569e-03 |   5.13%| unknown
  1.1s|     1 |     0 |   999 |     - |  2513k |   0 |  56 |  27 | 136 | 113 | 12 |   4 |   4 |-1.258987e-03 |-1.197569e-03 |   5.13%| unknown
  1.1s|     1 |     0 |  1001 |     - |  2518k |   0 |  56 |  27 | 137 | 114 | 13 |   4 |   4 |-1.258987e-03 |-1.197569e-03 |   5.13%| unknown
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  1.1s|     1 |     0 |  1001 |     - |  2520k |   0 |  56 |  27 | 137 | 114 | 13 |   4 |   6 |-1.258987e-03 |-1.197569e-03 |   5.13%| unknown
  1.1s|     1 |     0 |  1010 |     - |  2535k |   0 |  56 |  27 | 138 | 115 | 14 |   4 |   6 |-1.258987e-03 |-1.197569e-03 |   5.13%| unknown
  1.1s|     1 |     0 |  1017 |     - |  2545k |   0 |  56 |  27 | 139 | 116 | 15 |   4 |   6 |-1.258987e-03 |-1.197569e-03 |   5.13%| unknown

In [4]:
print("\nOptimal solution.")
fx_classical = problem.value
x_classical = x.value

x_classical = np.round(x_classical).astype(int)
sel_idx_classical = np.where(x_classical == 1)[0]
sel_tickers_classical = [TICKERS[i] for i in sel_idx_classical]

print(f"\n[Classical Optimum]")
print(f"Optimal QUBO cost f(x*) = {fx_classical:.6f}")
print(f"Selected {len(sel_idx_classical)} assets: {sel_tickers_classical}")
print(f"Bitstring (x*): {x_classical}")


Optimal solution.

[Classical Optimum]
Optimal QUBO cost f(x*) = -0.001198
Selected 4 assets: [np.str_('MSFT'), np.str_('JNJ'), np.str_('JPM'), np.str_('MS')]
Bitstring (x*): [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]


In [5]:
print("\nCalculating financial metrics for the optimal solution.")

w_classical = np.zeros(n)
w_classical[sel_idx_classical] = 1.0 / B

mu_day_classical = mu @ w_classical
var_day_classical = w_classical @ Sigma @ w_classical

mu_ann_classical = 252 * mu_day_classical
std_ann_classical = np.sqrt(252 * var_day_classical)

sharpe_classical = mu_ann_classical / std_ann_classical

print(f"\n[Classical Metrics (Equal Weights)]")
print(f"Expected annual return ≈ {mu_ann_classical:.2%}")
print(f"Annual volatility      ≈ {std_ann_classical:.2%}")
print(f"Sharpe ratio           ≈ {sharpe_classical:.3f}")


Calculating financial metrics for the optimal solution.

[Classical Metrics (Equal Weights)]
Expected annual return ≈ 47.71%
Annual volatility      ≈ 18.73%
Sharpe ratio           ≈ 2.548


In [6]:
np.savez(
    "cvxpy_results.npz",
    fx_cvxpy=np.array(fx_classical),
    mu_ann_cvxpy=np.array(mu_ann_classical),
    std_ann_cvxpy=np.array(std_ann_classical),
    sharpe_cvxpy=np.array(sharpe_classical),
    x_cvxpy=x_classical
)